In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install konlpy
!pip install flask-ngrok

     |████████████████████████████████| 958 kB 4.0 MB/s 
     |████████████████████████████████| 138 kB 65.3 MB/s 
     |████████████████████████████████| 356 kB 71.8 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 3.6 MB 31.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [13]:
#crawling
import heapq
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

#nlp
from konlpy.tag import Okt
from collections import Counter

#server 
from flask import Flask, url_for, redirect, render_template, request
from flask_ngrok import run_with_ngrok
import os

id_ = 'your id'
password = 'your password'

import pandas as pd

df_insa = pd.DataFrame(columns = ['text'])
df_jagwa = pd.DataFrame(columns = ['text'])

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'https://everytime.kr/login'

driver.get(url)

driver.find_element_by_name("userid").send_keys(id_)

driver.find_element_by_name("password").send_keys(password)

driver.find_element_by_tag_name("input").send_keys(Keys.RETURN)

time.sleep(2)
driver.find_element_by_xpath('//*[@id="submenu"]/div[1]/div[2]/ul/li[1]//a').click()

time.sleep(0.5)

res = driver.page_source
soup = BeautifulSoup(res,"html.parser")
data_name = soup.select('#container > div.wrap.articles > article > a > h2')
data_text = soup.select('#container > div.wrap.articles > article > a > p')

for name,text in zip(data_name,data_text):
    df_insa.loc[df_insa.shape[0]] = [name.get_text() + ' ' + text.get_text()]  
 
time.sleep(1)
def next_page(df):
    driver.find_element_by_css_selector('#container > div.wrap.articles > div.pagination > a.next').click()
    time.sleep(0.5)
    
    res = driver.page_source
    soup = BeautifulSoup(res,"html.parser")
    data_name = soup.select('#container > div.wrap.articles > article > a > h2')
    data_text = soup.select('#container > div.wrap.articles > article > a > p')   
    for name,text in zip(data_name,data_text):
        df.loc[df.shape[0]] = [name.get_text() + ' ' + text.get_text()]      

for page_roof in range(50):
    next_page(df_insa)
    time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="submenu"]/div[1]/div[2]/ul/li[2]//a').click()
time.sleep(0.5)

res = driver.page_source
soup = BeautifulSoup(res,"html.parser")
data_name = soup.select('#container > div.wrap.articles > article > a > h2')
data_text = soup.select('#container > div.wrap.articles > article > a > p')

for name,text in zip(data_name,data_text):
    df_jagwa.loc[df_jagwa.shape[0]] = [name.get_text() + ' ' + text.get_text()]  

for page_roof in range(50):
    next_page(df_jagwa)
    time.sleep(0.5)


driver.quit()

count_dic_insa = {}
count_dic_jagwa = {}

#text data processing and heapify

for text in df_insa.text:
  twitter = Okt()
  noun = twitter.nouns(str(text))
  count_dic_insa.update(Counter(noun))

noun_list_insa = Counter(count_dic_insa)
noun_list_insa = list(noun_list_insa.items())
for idx,noun in enumerate(noun_list_insa):
  if len(noun[0])==1:
    del noun_list_insa[idx]

for text in df_jagwa.text:
  twitter = Okt()
  noun = twitter.nouns(str(text))
  count_dic_jagwa.update(Counter(noun))

noun_list_jagwa = Counter(count_dic_jagwa)
noun_list_jagwa = list(noun_list_jagwa.items())
for idx,noun in enumerate(noun_list_jagwa):
  if len(noun[0])==1:
    del noun_list_jagwa[idx]

heapq.heapify(noun_list_insa)
heapq.heapify(noun_list_jagwa)


# insert into max heap
data_insa = []
max_heap_insa = []
for noun in noun_list_insa:
  heapq.heappush(max_heap_insa, (-noun[1], noun[1],noun[0]))

data_jagwa = []
max_heap_jagwa = []
for noun in noun_list_jagwa:
  heapq.heappush(max_heap_jagwa, (-noun[1], noun[1],noun[0]))

for i in range(10):
  trender = heapq.heappop(max_heap_insa)
  data_insa.append(trender[2])

for i in range(10):
  trender = heapq.heappop(max_heap_jagwa)
  data_jagwa.append(trender[2])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: De

In [9]:
data_insa

['화살', '정신', '과잠', '재수강', '점멸', '꽃', '끼리', '낭만', '동거', '맞팔']

In [10]:
data_jagwa

['해안', '가격', '비온', '졸업', '지역', '콜라', '가전자대', '갈비', '검정고시', '겨울']

In [19]:
# server

TEMPLATE_FOLDER = '/content/drive/MyDrive/algo/templates' # 임의의 경로
STATIC_FOLDER = '/content/drive/MyDrive/algo/static' # 임의의 경로
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)   

@app.route('/')
def index():
    return render_template('index.html', data_insa = {1:data_insa[0], 2:data_insa[1], 3:data_insa[2], 4:data_insa[3], 5:data_insa[4], 6:data_insa[5], 7:data_insa[6], 8:data_insa[7], 9:data_insa[8], 10:data_insa[9]}, data_jagwa = {1:data_jagwa[0], 2:data_jagwa[1], 3:data_jagwa[2], 4:data_jagwa[3], 5:data_jagwa[4], 6:data_jagwa[5], 7:data_jagwa[6], 8:data_jagwa[7], 9:data_jagwa[8], 10:data_jagwa[9]})

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a1ef-34-80-211-114.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Nov/2021 06:27:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 06:27:10] "GET /static/everytime.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 06:27:10] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 06:27:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 06:27:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 06:27:12] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 06:27:13] "GET /static/everytime.png HTTP/1.1" 200 -
